<a href="https://colab.research.google.com/github/RuoguLi0425/Robot-Learning/blob/main/Robot_Learning_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Important***

**Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/172081/pages/assignment-instructions) page on Courseworks2 to learn the workflow for completing this project.**

<ins>Important changes since Project 1 on the assignment instructions: </ins> When you are ready to submit, use "File -> Save and pin revision". Please name this revision "Grade me". We will only grade the revision that is correctly named. Late days will be applied according to this named revision.

# Intruduction

This project aims to demonstrate how neural networks can be used in a robotics setting. We will continue using the 2D maze environment introduced in Project 1 and learn to navigate an agent to a goal. However, since neural networks can be more powerful models than the ones we had access to previously, we can afford to make some changes to the 2D maze environment and make the problem more difficult.  This project consists of three parts. In Part I, you will be training a simple DNN, which will take as input the agent position and goal position. In Parts II and III, you will be training CNNs which take as input an image of the environment, with the agent the goal depicted on it.

<div>
<img src="https://github.com/roamlab/robot-learning-S2023/blob/main/project2/imgs/P1_side.png?raw=true" width="300"/>
</div>

The image above shows the simulation world. The "robot" (also called "agent") is shown by the green dot. The goal location is shown by the red square. The agent is required to navigate to the goal. **Unlike the previous project, the robot and the goal are spawned at random positions in the maze.** Also, the action space now contains all four directions: 'up', 'down', 'left' and 'right'. Another change is that, in addition to the obstacle map shown above, we introduce two new obstacle maps as shown below. However, these new maps will not be used until Part III.

<div>
<img src="https://github.com/roamlab/robot-learning-S2023/blob/main/project2/imgs/map1.png?raw=true" width="300"/>
<img src="https://github.com/roamlab/robot-learning-S2023/blob/main/project2/imgs/map2.png?raw=true" width="300"/>
<img src="https://github.com/roamlab/robot-learning-S2023/blob/main/project2/imgs/map3.png?raw=true" width="300"/>
</div>

We want to learn to navigate the agent by imitating demonstrations from an expert user. In all three parts, you will be using data collected by a human controlling the agent via a keyboard for training.

# Part 0. Project Setup

**Important: You need to follow these steps so that Colab can access the required data files and python source files.**

1. Download the zip file from the Google Drive link: https://drive.google.com/file/d/1uMY8x2kjW86d2VD21SVsTWtVa-zm_zSb/view?usp=share_link. Make sure you are logged into your LionMail account.
2. Click on the "Files" icon in the left sidebar. Then choose "Upload to session storage". Upload the zip file downloaded in the first step to Colab.
<div>
<img src="https://github.com/roamlab/robot-learning-S2023/blob/main/project2/imgs/guide_1.png?raw=true" width="300"/>
<img src="https://github.com/roamlab/robot-learning-S2023/blob/main/project2/imgs/guide_2.png?raw=true" width="300"/>
</div>
3. Run the cell below to unzip the zip file.

*You only need to download the zip file to your local machine (step 1) ONCE. But you need to do step 2 and step 3 EVERY TIME you connect to Colab.*

In [ ]:
# DO NOT CHANGE

# Make sure you have successfully uploaded the zip file to Colab before running the line below.
# Unzip the uploaded zip file
!unzip project2.zip -d /content/

Archive:  project2.zip
   creating: /content/mjcf/
  inflating: /content/mjcf/point_mass.xml  
   creating: /content/mjcf/common/
  inflating: /content/mjcf/common/skybox.xml  
  inflating: /content/mjcf/common/visual.xml  
  inflating: /content/mjcf/common/materials.xml  
  inflating: /content/mjcf/test_mjcf.xml  
  inflating: /content/dnn.py         
   creating: /content/imgs/
  inflating: /content/imgs/P1_side.png  
  inflating: /content/imgs/map1.png  
  inflating: /content/imgs/map3.png  
  inflating: /content/imgs/map2.png  
  inflating: /content/score_policy.py  
  inflating: /content/simple_maze.py  
  inflating: /content/data_utils.py  
   creating: /content/data/
  inflating: /content/data/map1.pkl  
  inflating: /content/data/all_maps.pkl  


In [ ]:
# DO NOT CHANGE

# Install required packages
!pip install pybullet==2.6.6 numpngw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pybullet: filename=pybullet-2.6.6-cp39-cp39-linux_x86_64.whl size=103262128 sha256=5eece61b44bdea26b27cd6f64c043c74d3133cd0a858a56e5591cb38cec6bca0
  Stored in directory: /root/.cache/pip/wheels/7e/ab/14/9235a3e8e4f3c31dcec4ea48039fc54139a3d05f3281c2dcfa
Successfully built pybullet


# Part I. Behavioral cloning with low dimensional data

This part is a natural extension of Part II in Project 1.

Learning the agent's policy here is the familiar classification problem, given that you will be provided with labeled examples from an expert. Each labeled example $i$ will contain a tuple of the form $(o, a)^i$, where $o$ represents an observation and $a$ represents the action taken by the expert given that observation. You must simply learn to imitate the expert, a process also known as behavioral cloning. While the action space is the same in all parts of the project, the observation space will be different.

We will be training a DNN policy to predict an action to be taken ('up', 'down', 'left', and 'right') based on the observation. **In Part I, the observation will contain the agent position and the current goal position.** (Because the goal is sampled randomly, the policy has to know the current goal to be reached.) The environment thus returns an observation array of size (4, ) where the agent position is contained in the first two axes and the current goal position is contained in the next two. **In Part I, the map that the robot is navigating is always the same.**

PyTorch and Tensorflow are two popular frameworks for building and training neural networks but for this class, we will be exclusively using PyTorch and you are allowed to use any of its features. A good starting point can be found [here](https://github.com/roamlab/robot-learning-S2023/blob/main/project2/dnn.py).

You will implement a class that inherits from `RobotPolicy` by providing implementations for the abstract methods from the class.



In [ ]:
# DO NOT CHANGE
# base class

import abc


class RobotPolicy(abc.ABC):

    @abc.abstractmethod
    def train(self, data):
        """
            Abstract method for training a policy.

            Args:
                data: a dict that contains X (key = 'obs') and y (key = 'actions').

                X is either rgb image (N, 64, 64, 3) OR  agent & goal pos (N, 4)

            Returns:
                This method does not return anything. It will just need to update the
                property of a RobotPolicy instance.
        """

    @abc.abstractmethod
    def get_action(self, obs):
        """
            Abstract method for getting action. You can do data preprocessing and feed
            forward of your trained model here.
            Args:
                obs: an observation (64 x 64 x 3) rgb image OR (4, ) positions

            Returns:
                action: an integer between 0 to 3
        """

In [ ]:
# Implement your solution for Part 1 below

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

torch.manual_seed(0)
np.random.seed(0)

class POSBCRobot(RobotPolicy):
    def __init__(self):

        self.model = nn.Sequential(
            nn.Linear(4, 128),  # Fully-connected layer with 4 input nodes and 128 output nodes
            nn.ReLU(),  # ReLU activation function
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 4)
        )

        self.LossFunction = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.003)

    def train(self, data):

        X = torch.tensor(data['obs'], dtype=torch.float32)
        y = torch.tensor(data['actions'], dtype=torch.long)

        iteration = 0
        while True:
            # Forward pass to obtain the logits of the model
            logits = self.model(X)
            # Calculate the loss using the specified loss function
            loss = self.LossFunction(logits, y)

            # Backward pass to calculate the gradients of the model parameters with respect to the loss
            self.optimizer.zero_grad()
            loss.backward()
            # Update the model parameters using the calculated gradients
            self.optimizer.step()

            # Print the loss every 30 iterations
            if iteration % 30 == 0:
                print(f"Iteration {iteration}: Loss={loss.item():.3f}")

            # Stop training if the loss is less than 0.10
            if loss.item() < 0.10:
                break

            iteration += 1


    def get_action(self, obs):

        obs = torch.tensor(obs, dtype=torch.float32)

        with torch.no_grad():
            logits = self.model(obs.unsqueeze(0))

        probs = torch.softmax(logits, dim=1)

        action = torch.multinomial(probs, num_samples=1).item()
        return action


## Evaluation and Grading

We will evaluate your model by simply having the agent follow the commands that it provides.  We will evaluate for 100 different randomly sampled starting positions and goals. For each goal, we roll out the trained policy for 50 steps. After the 50 steps, we will evaluate the closest distance to the goal the agent has ended up. If the agent reaches < 0.1 distance from the goal, the episode is ended before 50 steps and the minimum distance will be recorded as 0. The score is the fraction of the initial distance to goal covered by the agent averaged over 100 trials. Your final grade will be computed based on this score.

We will calculate the score using the formula :

```score = avg[(init_dist -  min_dist) / init_dist]```

We will auto-generate your grades using the code below. The grading of each part is separate from each other so you can get the grade right after each part is finished.

The total points of this assignment are 15. According to the difficulty level of each part, parts 1, 2, and 3 have 4, 5, 6 points respectively.

- Part 1: if your score >= 0.99, you will receive 4 / 4. Otherwise, your final grade will be score / 0.99 * 4.
- Part 2: if your score >= 0.95, you will receive 5 / 5. Otherwise, your final grade will be score / 0.95 * 5.
- Part 3: if your score >= 0.95, you will receive 6 / 6. Otherwise, your final grade will be score / 0.95 * 6.

The score function for each part provides two extra arguments to assist your debugging.

- gui: If this is set to True, you will save the behavior of the agents during evaluation as an animation file. This animation file can be visualized using the provided code below to help you understand the behavior of the agent. **Please set it to False before your submission as it will slow down evaluation.**
- model: If you provide a path to a saved model, the score function will not train from scratch but will instead load the save model. **Please set it to None before submission.** Any models you generate during runtime will be automatically deleted when disconnected. The grader will train the model from scratch.

In [ ]:
# DO NOT CHANGE
# Set up grading

import score_policy
import importlib
importlib.reload(score_policy)
from IPython.display import Image


part1_bound = 0.99
part2_bound = 0.95
part3_bound = 0.95

In [ ]:
# DO NOT CHANGE
# Getting the score and grade for Part 1

score1 = score_policy.score_pos_bc(policy=POSBCRobot(), gui=False, model=None)
grade1 = score1 / part1_bound * 4 if score1 < part1_bound else 4

print('\n---')
print(f'Part 1 Score: {score1}')
print(f'Part 1 Grade: {score1:.2f} / {part1_bound:.2f} * 4 = {grade1:.2f}')

Iteration 0: Loss=1.389
Iteration 30: Loss=0.927
Iteration 60: Loss=0.612
Iteration 90: Loss=0.525
Iteration 120: Loss=0.414
Iteration 150: Loss=0.361
Iteration 180: Loss=0.324
Iteration 210: Loss=0.295
Iteration 240: Loss=0.271
Iteration 270: Loss=0.250
Iteration 300: Loss=0.231
Iteration 330: Loss=0.216
Iteration 360: Loss=0.204
Iteration 390: Loss=0.191
Iteration 420: Loss=0.187
Iteration 450: Loss=0.172
Iteration 480: Loss=0.164
Iteration 510: Loss=0.156
Iteration 540: Loss=0.152
Iteration 570: Loss=0.143
Iteration 600: Loss=0.138
Iteration 630: Loss=0.131
Iteration 660: Loss=0.126
Iteration 690: Loss=0.121
Iteration 720: Loss=0.116
Iteration 750: Loss=0.109
Iteration 780: Loss=0.107
Iteration 810: Loss=0.102

---
Part 1 Score: 0.9914103909417972
Part 1 Grade: 0.99 / 0.99 * 4 = 4.00


In [ ]:
# Optionally, uncomment and run the code below if you have saved an animation (gui = True) that you want to visualize.

# Image(filename='part_1_anim.png', width=200, height=200)

# Part II. Behavioral cloning with visual observations

In this part, you asked to do a similar task as Part I, **but the observations will be RGB image observations of the world**, similar to the ones you used to do localization in Part III of Project 1. To process the RGB image you will be implementing a CNN using PyTorch.  [The official PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) is a good starting point. As in Part I, the map that the robot is navigating is always the same. **This means that your model really only has to learn how to figure out where the robot and the goal are located, and how to navigate around a fixed set of obstacles.**

All requirements from your code, as well as the evaluation method, are unchanged compared to Part I. The only difference is the nature of the observation that is provided to you.

In [ ]:
# Implement your solution for Part 2 below

import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset

torch.manual_seed(0)
np.random.seed(0)

class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 3, 1, 1)
        self.pool = nn.MaxPool2d(4)
        self.fc1 = nn.Linear(8 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 4)
        self.relu = nn.ReLU()


    def forward(self, x):
        x = self.conv1(x)
        if x.shape[-1] % 4 != 0:
            raise ValueError("The output shape of the first convolutional layer must be divisible by 4.")

        x = self.relu(x)
        x = self.pool(x)
        x = x.reshape(-1, 2048)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x

class RGBBCRobot1(RobotPolicy):
    """ Implement solution for Part2 below """

    def __init__(self):
        super(RGBBCRobot1, self).__init__()
        self.CNN = CNN_Model()
    # Add a comment here to explain the early stopping criterion
    def train(self, data):
        for key, val in data.items():
            print(key, val.shape)
        print("Using dummy solution for RGBBCRobot1")

        X = torch.tensor(data['obs'], dtype=torch.float32).permute(0, 3, 1, 2)
        Y = torch.tensor(data['actions'], dtype=torch.int64).flatten().reshape(-1, 1)

        batch_size = 20
        train_set = torch.utils.data.TensorDataset(X, Y)
        train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=0)

        optimizer = Adam(self.CNN.parameters(), lr=0.003)
        LossF = nn.CrossEntropyLoss()

        iteration = 0
        train_loss = 1
        max_iteration = 100
        # Stop training if the average training loss is less than 0.2 or the maximum number of iterations is reached
        while train_loss > 0.2 and iteration < max_iteration:
            iteration += 1
            train_loss = 0
            self.CNN.train()

            lr = 0.003

            for train_data, train_label in train_loader:
                optimizer.zero_grad()
                train_label_predict = self.CNN(train_data)
                train_label = train_label.reshape(-1)
                loss = LossF(train_label_predict, train_label)  # Use LossF instead of loss_fn
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            # This is to prevent overfitting and reduce training time
            train_loss /= len(train_loader)
            if iteration % 1 == 0:
                print(f"Iteration: {iteration}, Loss Function Value = {train_loss:.3f}")

    def get_action(self, obs):
        if not isinstance(obs, np.ndarray):
            raise ValueError("The input observation must be a numpy array.")
        if obs.ndim != 3 or obs.shape[-1] != 3:
            raise ValueError("The input observation must be a RGB image with shape (height, width, 3).")
        obs = np.expand_dims(obs, axis=0)
        obs = obs.transpose(0, 3, 1, 2)
        obs_t = torch.tensor(obs).float()
        self.CNN.eval()
        with torch.set_grad_enabled(False):
            action = self.CNN(obs_t)
        predicted = torch.max(action, 1)[1].item()
        return predicted


## Evaluation and Grading

In [ ]:
# DO NOT CHANGE
# Getting the score and grade for Part 2

score2 = score_policy.score_rgb_bc1(policy=RGBBCRobot1(), gui=False, model=None)
grade2 = score2 / part2_bound * 5 if score2 < part2_bound else 5

print('\n---')
print(f'Part 2 Score: {score2}')
print(f'Part 2 Grade: {score2:.2f} / {part2_bound:.2f} * 5 = {grade2:.2f}')

actions (4000,)
obs (4000, 64, 64, 3)
Using dummy solution for RGBBCRobot1
Iteration: 1, Loss Function Value = 1.074
Iteration: 2, Loss Function Value = 0.951
Iteration: 3, Loss Function Value = 0.931
Iteration: 4, Loss Function Value = 0.914
Iteration: 5, Loss Function Value = 0.896
Iteration: 6, Loss Function Value = 0.877
Iteration: 7, Loss Function Value = 0.805
Iteration: 8, Loss Function Value = 0.477
Iteration: 9, Loss Function Value = 0.297
Iteration: 10, Loss Function Value = 0.199

---
Part 2 Score: 0.9827320877869944
Part 2 Grade: 0.98 / 0.95 * 5 = 5.00


In [ ]:
# Optionally, uncomment and run the code below if you have saved an animation (gui = True) that you want to visualize.

# Image(filename='part_2_anim.png', width=200, height=200)

# Part III. Behavioral cloning with visual observations - multiple maps

This part is the same as  Part II except that it is trained and tested differently. **The training set involves expert demonstrations for the two new obstacle maps. And while testing, for each trial, a different obstacle map is randomly selected.** This means that your model has to learn how to reason about what an obstacle is, and how to go around it, based on nothing more than an image. The main objective of this part is to show that, when using a CNN, it is possible for a model to achieve this. The evaluation method for this part is the same as Part I and II.

In [ ]:
# Implement your solution for Part 3 below

import numpy as np
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import OrderedDict


torch.manual_seed(0)
np.random.seed(0)

class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 3, 1, 1)
        self.pool = nn.MaxPool2d(4)
        self.fc1 = nn.Linear(8 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 4)
        self.relu = nn.ReLU()


    def forward(self, x):
        x = self.conv1(x)
        if x.shape[-1] % 4 != 0:
            raise ValueError("The output shape of the first convolutional layer must be divisible by 4.")
        x = self.relu(x)
        x = self.pool(x)
        x = x.reshape(-1, 2048)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()

        self.conv1 = nn.Conv2d(3, 4, 3, 1, 1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(4, 16, 3, 1, 1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)

        self.fc1 = nn.Linear(16 * 16 * 16, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 32)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(32, 4)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.reshape(-1, 16 * 16 * 16)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.relu4(x)
        x = self.fc3(x)
        return x

class RGBBCRobot2(RobotPolicy):
    """ Implement solution for Part2 below """

    def __init__(self):
        super(RGBBCRobot2, self).__init__()
        self.CNN = CNN_Model()

    def train(self, data):
        for key, val in data.items():
            print(key, val.shape)
        print("Using dummy solution for RGBBCRobot2")

        X = torch.tensor(data['obs'], dtype=torch.float32).permute(0, 3, 1, 2)
        Y = torch.tensor(data['actions'], dtype=torch.int64).flatten().reshape(-1, 1)

        batch_size = 30
        train_set = torch.utils.data.TensorDataset(X, Y)
        train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=0)

        # optimizer = optim.Adam(self.CNN.parameters(), lr=0.003)
        optimizer = optim.SGD(self.CNN.parameters(), lr=0.003, momentum=0.9)
        loss_f = nn.CrossEntropyLoss()

        iteration = 0
        train_loss = 1
        max_iteration = 10


        while train_loss > 0.1 and iteration < max_iteration:
            iteration += 1
            train_loss = 0
            self.CNN.train()

            lr = 0.003

            for train_data, train_label in train_loader:
                optimizer.zero_grad()
                train_label_predict = self.CNN(train_data)
                train_label = train_label.reshape(-1)
                loss = loss_f(train_label_predict, train_label)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            train_loss /= len(train_loader)

            if iteration % 1 == 0:
                print(f"Iteration: {iteration}, Loss Function Value = {train_loss:.3f}")

    def get_action(self, obs):
        if not isinstance(obs, np.ndarray):
            raise ValueError("The input observation must be a numpy array.")
        if obs.ndim != 3 or obs.shape[-1] != 3:
            raise ValueError("The input observation must be a RGB image with shape (height, width, 3).")
        obs = np.expand_dims(obs, axis=0)
        obs = obs.transpose(0, 3, 1, 2)
        obs_t = torch.tensor(obs).float()
        self.CNN.eval()
        with torch.set_grad_enabled(False):
            action = self.CNN(obs_t)
        predicted = torch.max(action, 1)[1].item()
        return predicted


## Evaluation and Grading


In [ ]:
# DO NOT CHANGE
# Getting the score and grade for Part 3

score3 = score_policy.score_rgb_bc2(policy=RGBBCRobot2(), gui=False, model=None)
grade3 = score3 / part3_bound * 6 if score3 < part3_bound else 6

print('\n---')
print(f'Part 3 Score: {score3}')
print(f'Part 3 Grade: {score3:.2f} / {part3_bound:.2f} * 6 = {grade3:.2f}')

actions (12000,)
obs (12000, 64, 64, 3)
Using dummy solution for RGBBCRobot2
Iteration: 1, Loss Function Value = 1.375
Iteration: 2, Loss Function Value = 1.343
Iteration: 3, Loss Function Value = 1.326
Iteration: 4, Loss Function Value = 1.183
Iteration: 5, Loss Function Value = 1.083
Iteration: 6, Loss Function Value = 1.036
Iteration: 7, Loss Function Value = 0.815
Iteration: 8, Loss Function Value = 0.532
Iteration: 9, Loss Function Value = 0.365
Iteration: 10, Loss Function Value = 0.279

---
Part 3 Score: 0.9570037915956159
Part 3 Grade: 0.96 / 0.95 * 6 = 6.00


In [ ]:
# Optionally, uncomment and run the code below if you have saved an animation (gui = True) that you want to visualize.

# Image(filename='part_3_anim.png', width=200, height=200)

# Other Requirements and Hints

- **Training time**: To keep auto-grading feasible, your total training time must be strictly under 3 mins, 15mins, and 10 mins for parts 1, 2, and 3. These time budgets are more than enough to achieve full credits on this project. Note that longer training time does not necessarily mean higher performance because of overfitting. The faster your network trains the better!
- **Memory usage**: Make sure your code does not require too much memory. The required amount of RAM for this assignment should not be more than 8GB.
- **NO GPU**: No GPU is required or allowed for this assignment.
- **Reproducibility**: We have ensured that the randomness of the environment is deterministic. To get reproducible scores you must ensure your model training and prediction are also reproducible. The randomly initialized weights of the neural network should be made repeatable using seeding. You can add PyTorch seeding method below and see [PyTorch Reproducibility](https://pytorch.org/docs/stable/notes/randomness.html) to learn more.
  ```
  import torch
  torch.manual_seed(0)
  ```
- **Classifier**: In all the parts we are training a neural network to solve a classification problem and it is important to use a reasonable loss function. For example, the MSE (mean squared classification) error has drawbacks related to sensitivity. Cross entropy loss usually has good performance for classification tasks and you can find the documentation for it [here](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) and is further explained below. However, note that you are free to use any loss function you like.
  - Cross entropy is a concept from information theory which is defined for two probability distributions. Cross entropy is minimum when the two distributions involved are the same and this is the property that makes it useful as a loss function in the context of machine learning. The idea is to minimize the cross entropy between the prediction distribution and the label distribution. For our case where we are training a neural network for classification, we can have the network output a score for each action. Cross entropy can be computed from these scores by converting to probability values (using softmax) and comparing it with the label distribution. The label distribution is obtained simply by assigning a probability of 1 to ground truth action and 0 to all other actions. Once trained, the best action can found by just choosing the action with the highest probability (i.e., the highest score) as predicted by the network.
- **Optimizer**: While it is possible to use a simple optimizer to achieve the desired accuracy, the training time can be quite high. There exist a number of optimizers implemented in PyTorch that have much faster convergence.
- **Parameter tuning**: Keep your architectures simple and slowly add complexity (more layers/kernels) to improve accuracy. Remember "To Err is Human" and the expert data (collected by a human) that you are training on is not perfect. Having a 100% training accuracy (very small training loss) might not be the best for achieving the highest score. So make sure your model does not overfit during training.
- **PyTorch input shape**: Notice that the expected input shape to CONV2D in PyTorch is (N, C, H, W), where N is the batch size, C is the number of channels, H is the image height and W is the image width. You will need to switch axes for the incoming images in order for them to be correctly passed to the first convolution layer.